# Computer Vision Playground: Needle
## Author: Dimitri Lezcano
This is an image processing playground Python Notebook in order to test image processing solutions.

## Imports

In [ ]:
%matplotlib inline
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from scipy.optimize import Bounds, minimize
from scipy.interpolate import BPoly

# pltting tools
from mpl_toolkits.mplot3d import Axes3D  # 3D plotting @UnusedImport

# NURBS
from geomdl import fitting
from geomdl.visualization import VisMPL

# python debugging
from IPython.core.debugger import set_trace

# custom code
from stereo_needle_proc import *
from image_processing import find_param_along_bspline
from BSpline1D import BSpline1D
from BSpline3D import BSpline3D

## Load Image Files

In [ ]:
# directory settings
stereo_dir = "../data/"
needle_dir = stereo_dir + "needle_examples/"  # needle insertion examples directory
grid_dir = stereo_dir + "grid_only/"  # grid testing directory
valid_dir = stereo_dir + "stereo_validation_jig/"  # validation directory

curvature_dir = glob.glob( valid_dir + 'k_*/' )  # validation curvature directories

# load matlab stereo calibration parameters
stereo_param_dir = "../calibration/Stereo_Camera_Calibration_10-23-2020"
stereo_param_file = stereo_param_dir + "/calibrationSession_params-error_opencv-struct.mat"
stereo_params = load_stereoparams_matlab( stereo_param_file )

In [ ]:
# image files
img_num = 0
left_file = curvature_dir[0] + f"left-{img_num:04d}.png"
right_file = curvature_dir[0] + f"right-{img_num:04d}.png"

# load the images
left_img = cv.imread(left_file, cv.IMREAD_ANYCOLOR)
left_img = cv.cvtColor(left_img, cv.COLOR_BGR2RGB)
right_img = cv.imread(right_file, cv.IMREAD_ANYCOLOR)
right_img = cv.cvtColor(right_img, cv.COLOR_BGR2RGB)

# show the images
print(left_img.shape, right_img.shape)
plt.figure(figsize=(18,12))
plt.imshow(imconcat(left_img, right_img, [255,0, 0]))
plt.title('Raw image')

## Image Processing
Image processing functions to test.

In [ ]:
def roi_image2(reg_of_int, image_size):
    
    roi_bool = np.zeros(image_size[:2], dtype=bool) # initialization
    
    tl, br = reg_of_int # unpack

    # check for negative indices
    for i, (tl_i, br_i) in enumerate(zip(tl, br)):
        if tl_i < 0:
            tl[i] = image_size[i] + tl_i + 1
            
        if br_i < 0:
            br[i] = image_size[i] + br_i + 1
            
    # for
    
    roi_bool[tl[0]:br[0], tl[1]:br[1]] = True
    
    return roi_bool
    
# roi_image2


# USE THIS ONE!
def roi_image3(reg_of_int, image_size):
    roi_bool = np.ones(image_size[:2], dtype=bool)
    
    roi_bool = roi(roi_bool, reg_of_int).astype(bool)
    
    return roi_bool
    
# roi_image3

# USE THIS ONE!
def blackout_image3(bo_regions: list, image_size):
    bo_bool = np.ones(image_size[:2], dtype=bool)
    
    bo_bool = blackout_regions(bo_bool, bo_regions)
    
    return bo_bool

# blackout_image3
    
    
a, b = [[0,0], [1,2]]
region = [a, b]
img_shape = (3,5,10)

roi_img = roi_image([a,b], img_shape)
method2 = roi_image2([a,b], img_shape)
method3 = roi_image3([a,b], img_shape)

bo_img = blackout_image([], img_shape)
bo3 =  blackout_image3([], img_shape)

print('roi\n', roi_img)
print('roi2:\n', method2)
print('roi3:\n', method3)

print('equals?:', np.all(method2 == method3))
print()

print('bor\n', bo_img)
print('bor3:\n',bo3)

print('not equal?', np.all(bo3 != method3), '\n', bo3 != method3)

In [ ]:
def imgproc_jig2( left_img, right_img,
                         bor_l:list = [], bor_r:list = [],
                         roi_l:tuple = (), roi_r:tuple = (),
                         proc_show: bool = False ):
    ''' wrapper function to process the left and right image pair for needle
        centerline identification
        
     '''
    
    # convert to grayscale if not already
    if left_img.ndim > 2:
        left_img = cv.cvtColor( left_img, cv.COLOR_BGR2GRAY )

    # if
    if right_img.ndim > 2:
        right_img = cv.cvtColor( right_img, cv.COLOR_BGR2GRAY )

    # if
    
#     # image processing 1
#     left_canny, right_canny = canny(left_img, right_img, lo_thresh=150, hi_thresh=200)
#     left_roi1 = roi( left_canny, roi_l, full = True )
#     right_roi1 = roi( right_canny, roi_r, full = True )
    
#     left_thresh_bo1 = blackout_regions( left_roi1, bor_l )
#     right_thresh_bo1 = blackout_regions( right_roi1, bor_r )
    
#     left_close1, right_close1 = bin_close( left_thresh_bo1, right_thresh_bo1, ksize = ( 7, 7 ) )
    
    # start the image qprocessing
    left_thresh, right_thresh = thresh( left_img, right_img, thresh=30 )
    
    left_roi = roi( left_thresh, roi_l, full = True )
    right_roi = roi( right_thresh, roi_r, full = True )
    
    left_thresh_bo = blackout_regions( left_roi, bor_l )
    right_thresh_bo = blackout_regions( right_roi, bor_r )
        
    left_tmed, right_tmed = median_blur( left_thresh_bo, right_thresh_bo, ksize = 7 )
    
    left_close, right_close = bin_close( left_tmed, right_tmed, ksize = ( 7, 7 ) )
    
    left_open, right_open = bin_open( left_close, right_close, ksize = ( 3, 3 ) )
    
    left_dil, right_dil = bin_dilate( left_close, right_close, ksize = ( 3, 3 ) )
    
    left_skel, right_skel = skeleton( left_dil, right_dil )
    
    # get the contours ( sorted by length)
    conts_l, conts_r = contours( left_skel, right_skel )
    
    # fit a bspline to the contours
    conts_l_filt = [c for c in conts_l if len(c) > 0]
    conts_r_filt = [c for c in conts_r if len(c) > 0]
    pts_l = np.unique(np.vstack(conts_l_filt).squeeze(), axis=0)
    pts_r = np.unique(np.vstack(conts_r_filt).squeeze(), axis=0)
    bspline_l = BSpline1D(pts_l[:,0], pts_l[:,1], k=3)
    bspline_r = BSpline1D(pts_r[:,0], pts_r[:,1], k=3)
        
    # grab all of the bspline points
    s = np.linspace(0, 1, 200)
    bspline_pts_l = np.vstack((bspline_l.unscale(s), bspline_l(bspline_l.unscale(s)))).T
    bspline_pts_r = np.vstack((bspline_r.unscale(s), bspline_r(bspline_r.unscale(s)))).T
    
    if proc_show:
        plt.ion()
        
        plt.figure(figsize=(18,12))
        plt.imshow( imconcat( left_thresh, right_thresh, 150 ), cmap = 'gray' )
        plt.title( 'adaptive thresholding' )
        
        plt.figure(figsize=(18,12))
        plt.imshow( imconcat( left_roi, right_roi, 150 ), cmap = 'gray' )
        plt.title( 'roi: after thresholding' )
        
        plt.figure(figsize=(18,12))
        plt.imshow( imconcat( left_thresh_bo, right_thresh_bo, 150 ), cmap = 'gray' )
        plt.title( 'region suppression: roi' )
        
        plt.figure(figsize=(18,12))
        plt.imshow( imconcat( left_tmed, right_tmed, 150 ), cmap = 'gray' )
        plt.title( 'median filtering: after region suppression' )
        
        plt.figure(figsize=(18,12))
        plt.imshow( imconcat( left_close, right_close, 150 ), cmap = 'gray' )
        plt.title( 'closing: after median' )
        
        plt.figure(figsize=(18,12))
        plt.imshow( imconcat( left_open, right_open, 150 ), cmap = 'gray' )
        plt.title( 'opening: after closing' )

        plt.figure(figsize=(18,12))
        plt.imshow( imconcat( left_open, right_open, 150 ), cmap = 'gray' )
        plt.title( 'dilation: after closing' )
        
        plt.figure(figsize=(18,12))
        plt.imshow( imconcat( left_skel, right_skel, 150 ), cmap = 'gray' )
        plt.title( 'skeletization: after dilation' )
        
        cont_left = left_img.copy().astype( np.uint8 )
        cont_right = right_img.copy().astype( np.uint8 )
        
        cont_left = cv.cvtColor( cont_left, cv.COLOR_GRAY2RGB )
        cont_right = cv.cvtColor( cont_right, cv.COLOR_GRAY2RGB )
        
        cv.drawContours( cont_left, conts_l_filt, -1, ( 255, 0, 0 ), 6 )
        cv.drawContours( cont_right, conts_r_filt, -1, ( 255, 0, 0 ), 6 )
        
        cv.drawContours( cont_left, conts_l_filt, 0, ( 0, 255, 0 ), 3 )
        cv.drawContours( cont_right, conts_r_filt, 0, ( 0, 255, 0 ), 3 )
        
        plt.figure(figsize=(18,12))
        plt.imshow( imconcat( cont_left, cont_right, 150 ), cmap = 'gray' )
        plt.title( 'contours' )
        
        impad = 20
        bspl_left_img = cv.cvtColor(left_img.copy().astype(np.uint8), cv.COLOR_GRAY2RGB)
        bspl_right_img = cv.cvtColor(right_img.copy().astype(np.uint8), cv.COLOR_GRAY2RGB)
        
        plt.figure(figsize=(18,12))
        plt.imshow(imconcat(bspl_left_img, bspl_right_img, [0, 0, 255],pad_size=impad))
        plt.plot(bspline_pts_l[:,0], bspline_pts_l[:,1], 'r-')
        plt.plot(bspline_pts_r[:,0]  + left_img.shape[1] + impad, bspline_pts_r[:,1], 'r-')
        plt.title('bspline fits')
        
        
    # if
    
    return left_skel, right_skel, [np.expand_dims(bspline_pts_l, axis=1)], [np.expand_dims(bspline_pts_r, axis=1)], conts_l_filt, conts_r_filt
    
# imgproc_jig2


In [ ]:
def match_bsplines(img_l, img_r, bspline_l, bspline_r, win_size, threshold=40):
    img_lg = cv.cvtColor(img_l, cv.COLOR_BGR2GRAY)
    img_rg = cv.cvtColor(img_r, cv.COLOR_BGR2GRAY)

    img_lt, img_rt = thresh(img_lg, img_rg, thresh=threshold)
    
    img_lgn = (img_lg - np.mean(img_lg))/np.std(img_lg)
    img_rgn = (img_rg - np.mean(img_rg))/np.std(img_rg)
    
    s = np.linspace(0, 1, 200)
    
    if win_size % 2 != 1:
        raise ValueError('win_size must be odd.')
        
    # if
    
    # window points
    dp = np.arange(win_size) - win_size//2
    dx, dy = np.meshgrid(dp, dp) # window poitns
    dpts = np.hstack( (dx.reshape(-1,1), dy.reshape(-1,1)) ).reshape(1, -1, 2)
    
    def cost_fn(s2_0, img1, img2, bspline1, bspline2):
        nonlocal dpts
        s1 = np.linspace(0, 1, 200)
        s2 = np.linspace(s2_0, 1, 200)
            
        # grab the image pixel values
        x1 = bspline1.unscale(s1)
        pts1 = np.vstack((x1, bspline1(x1))).T.reshape(-1,1,2).astype(int)
        win_pts1 = pts1 + dpts
        img1_v = img1[win_pts1[:,:,0], win_pts1[:,:,1]]
        
        x2 = bspline2.unscale(s2)
        pts2 = np.vstack((x2, bspline2(x2))).T.reshape(-1,1,2).astype(int)
        win_pts2 = pts2 + dpts
        img2_v = img2[win_pts2[:,:,0], win_pts2[:,:,1]]
        
        # perform normalized x-correlation
        img1_vn = np.sqrt( np.sum(img1_v**2, axis=1) )
        img2_vn = np.sqrt( np.sum(img2_v**2, axis=1) )
        normxcorr = np.sum(img1_v*img2_v, axis=1)/img1_vn/img2_vn
        
        
        cost = np.sum( 1 - normxcorr )
        
        cost = np.sum((img1_v - img2_v)**2)
        
        return cost

    # cost_fn
    
    
    if bspline_l.unscale(1) < bspline_r.unscale(1):
        sl = np.linspace(0, 1, 200)
        xl = bspline_l.unscale(sl)
        pts_l = np.vstack((xl, bspline_l(xl))).T
        
        bnds = Bounds(0, .9)
        min_args = (img_lgn, img_rgn, bspline_l, bspline_r)
        res = minimize(cost_fn, [.5], args=min_args, method='SLSQP', bounds=bnds)
        
        sl_opt = 0
        sr_opt = res.x[0]
        
        # grab pts_r
        sr = np.linspace(sr_opt, 1, 200)
        xr = bspline_r.unscale(sr)
        pts_r = np.vstack((xr, bspline_r(xr))).T
        
        
    # if
    
    return pts_l, pts_r, sl_opt, sr_opt

# match_bsplines
    
    
    

Test Grabbing Image Points 

In [ ]:
gray = cv.cvtColor(left_img, cv.COLOR_BGR2GRAY)
win_size = 5
dp = np.arange(win_size) - win_size//2
dx, dy = np.meshgrid(dp, dp)
dpts = np.hstack((dx.reshape(-1,1), dy.reshape(-1,1))).reshape(1,-1,2)

img_pt = np.array([100, 200, 300, 400]).reshape(-1,1,2)
img_pts = img_pt + dpts
vals = gray[img_pts[:,:,0], img_pts[:,:,1]]
print(vals.shape, dpts.shape, img_pt.shape, img_pts.shape, img_pts.shape, vals[0].shape)

print(np.sum(vals*vals, axis=1)/np.sum(vals**2, axis=1))

print(vals[0])
print(np.sum(vals[0]*vals[0])/(vals[0]**2).sum())
print(np.sum(vals[0]*vals[0])/np.linalg.norm(vals[0])**2)

print(np.linalg.norm(vals, axis=1, ord=2)**2, np.sum(vals*vals, axis=1))


## Testing Image Processing

### Region-of-Interest Mappings

### Image Processing Steps

In [ ]:
rois_lr = np.load(curvature_dir[0] + 'rois_lr.npy')
roi_l = tuple(rois_lr[img_num, :, 0:2])
roi_r = tuple(rois_lr[img_num, :, 2:4])
*_, cont_l, cont_r = imgproc_jig2(left_img, right_img, roi_l = roi_l, roi_r = roi_r,
                                 proc_show=False)

In [ ]:
pts_l = np.concatenate(cont_l).squeeze()
pts_r = np.concatenate(cont_r).squeeze()

print(pts_l.shape, pts_r.shape)
fig = plt.figure()
ax1 = fig.add_subplot(1,2,1)
ax1.plot(pts_l[:,0], pts_l[:,1], '.')

ax2 = fig.add_subplot(1,2,2)
ax2.plot(pts_r[:,0], pts_r[:,1], '.')

In [ ]:
# try using `centerline_from_contours` function
len_thresh = -1
out_thresh = 1.5
n_neigh = 10
bspl_k = -1

ct_pts_l, bspline_l = centerline_from_contours(cont_l, len_thresh = len_thresh,
                                               outlier_thresh=out_thresh, num_neigbors=n_neigh,
                                               bspline_k = bspl_k)

# try using `centerline_from_contours` function
ct_pts_r, bspline_r = centerline_from_contours(cont_r, len_thresh = len_thresh,
                                               outlier_thresh=out_thresh, num_neigbors=n_neigh,
                                               bspline_k = bspl_k)

left_match, right_match = stereomatch_needle([ct_pts_l], [ct_pts_r])

fig = plt.figure(figsize=(12,6))
ax1 = fig.add_subplot(1,2,1)
ax1.plot(pts_l[:,0], pts_l[:,1], 'b.', markersize=16)
ax1.plot(ct_pts_l[:,0], ct_pts_l[:,1], 'r.')
ax1.plot(left_match[:,0], left_match[:,1], 'g.')

ax2 = fig.add_subplot(1,2,2)
ax2.plot(pts_r[:,0], pts_r[:,1], 'b.', markersize=16)
ax2.plot(ct_pts_r[:,0], ct_pts_r[:,1], 'r.')
ax2.plot(right_match[:,0], right_match[:,1], 'g.')

# stereo match the points
# left_match, right_match = stereomatch_needle(ct_pts_l, ct_pts_r, bspline_l=bspline_l, bspline_r=bspline_r)

# print(left_match.shape[0], right_match.shape[0])

option = 2

if (bspline_l is not None) and (bspline_r is not None):
    
    if option == 1:
        s = np.array([0, 1])
        x_l = bspline_l.unscale(s)
        x_r = bspline_r.unscale(s)
        al_l = bspline_l.arclength(x_l[0], x_l[1])
        al_r = bspline_r.arclength(x_r[0], x_r[1])
        s2 = 0.98
        al_l2 = bspline_l.arclength(x_l[0], bspline_l.unscale(s2))
        print('arclength:', al_l, al_r)

        # set-up optimization problem
        bnds = Bounds( 0, 1 )
        xr_tip = x_r[1]
        arclen_r = lambda s: bspline_r.arclength( bspline_r.unscale( s ), xr_tip )
        cost_fn_sc = lambda s: np.abs( bspline_l._scale(al_l) -  bspline_r._scale(arclen_r( s )) )
        cost_fn_t = lambda s: np.abs( al_l -  arclen_r( s ) )
        cost_fn = cost_fn_sc

        res = minimize( cost_fn,[.9], method = 'SLSQP', bounds = bnds )

        print('scaled result:', res.x[0], cost_fn(res.x))

        sr_opt_sc = res.x[0]

        sr = np.linspace(sr_opt_sc, 1, 200)
        left_match = ct_pts_l.copy()
        xr = bspline_r.unscale(sr)
        right_match = np.vstack((xr, bspline_r(xr))).T

        print(xr.shape, right_match.shape)

        ax1.plot(left_match[:,0], left_match[:,1], 'g.')
        ax2.plot(right_match[:,0], right_match[:,1], 'g.')

        conts_l = [np.vstack((left_match.astype(int), np.flip(left_match, axis=0).astype(int)))]
        conts_r = [np.vstack((right_match.astype(int), np.flip(right_match, axis=0).astype(int)))]

        left_cont = left_img.copy()
        right_cont = right_img.copy()

        cv.drawContours(left_cont, conts_l, -1, [0, 255, 0], 2)
        cv.drawContours(right_cont, conts_r, -1, [0, 255, 0], 2)


        plt.figure(figsize=(12,8))
        plt.imshow(imconcat(left_cont, right_cont, [0, 0, 255]))
        plt.title('scaled')

        # try the other cost function
        cost_fn = cost_fn_t

        res = minimize( cost_fn,[.9], method = 'SLSQP', bounds = bnds )

        print('unscaled result:', res.x[0], cost_fn(res.x))

        sr_opt_t = res.x[0]

        sr = np.linspace(sr_opt_t, 1, 200)
        left_match = ct_pts_l.copy()
        xr = bspline_r.unscale(sr)
        right_match = np.vstack((xr, bspline_r(xr))).T

        print(xr.shape, right_match.shape)

        ax1.plot(left_match[:,0], left_match[:,1], 'y.')
        ax2.plot(right_match[:,0], right_match[:,1], 'y.')
        ax2.legend(['img', 'func', 'scaled match', 'unscaled match'])


        conts_l = [np.vstack((left_match.astype(int), np.flip(left_match, axis=0).astype(int)))]
        conts_r = [np.vstack((right_match.astype(int), np.flip(right_match, axis=0).astype(int)))]

        left_cont = left_img.copy()
        right_cont = right_img.copy()

        cv.drawContours(left_cont, conts_l, -1, [0, 255, 0], 2)
        cv.drawContours(right_cont, conts_r, -1, [0, 255, 0], 2)


        plt.figure(figsize=(12,8))
        plt.imshow(imconcat(left_img, right_img, [0, 0, 255]))
        plt.title('image')

        plt.figure(figsize=(12,8))
        plt.imshow(imconcat(left_cont, right_cont, [0, 0, 255]))
        plt.title('unscaled')

    if option == 2:
        left_match, right_match, sl_opt, sr_opt = match_bsplines(left_img, right_img,
                                                                 bspline_l, bspline_r, 51)
        
        print(sl_opt, '|', sr_opt)
        
        ax1.plot(left_match[:,0], left_match[:,1], 'g.')
        ax2.plot(right_match[:,0], right_match[:,1], 'g.')


# x_rb, err = find_param_along_bspline(bspline_r, x_r[1], al_l, lb=x_r[0], ub=x_r[1])

# mask = ct_pts_r[:,0] > x_rb


In [ ]:
al_l = bspline_l.arclength(bspline_l.unscale(0), bspline_l.unscale(1))
al_r = bspline_r.arclength(bspline_r.unscale(0), bspline_r.unscale(1))
print((bspline_l.qmax - bspline_l.qmin)/(bspline_r.qmax - bspline_r.qmin))
print(al_l/al_r)

In [ ]:
cont_l_min = np.reshape(cont_l[-1], (-1, 2))
cont_r_min = np.reshape(cont_r[-1], (-1, 2))

mean_l = np.mean(cont_l_min, axis=0)
mean_r = np.mean(cont_r_min, axis=0)

print('difference', mean_r - mean_l)

print(len(cont_l), cont_l[-1].shape, cont_l_min.shape)
print(len(cont_r), cont_r[-1].shape, cont_r_min.shape)

fig = plt.figure(figsize=(12,6))
ax1 = fig.add_subplot(1,2,1)
ax1.plot(pts_l[:,0], pts_l[:,1], 'b.')
ax1.plot(ct_pts_l[:,0], ct_pts_l[:,1], 'r.')
ax1.plot(cont_l_min[:,0], cont_l_min[:,1], 'k.')


ax2 = fig.add_subplot(1,2,2)
ax2.plot(pts_r[:,0], pts_r[:,1], 'b.')
ax2.plot(ct_pts_r[:,0], ct_pts_r[:,1], 'r.')
ax2.plot(cont_r_min[:,0], cont_r_min[:,1], 'k.')


In [ ]:
from sklearn.cluster import KMeans
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
clf = LocalOutlierFactor(n_neighbors=20, contamination='auto')
clf.fit_predict(pts_r)
scores = clf.negative_outlier_factor_

mask = np.abs(-1-scores) < 0.5

pts_r_in = pts_r[mask]

plt.figure()
plt.plot(pts_r[:,0], pts_r[:,1],'b.')
plt.plot(pts_r_in[:,0], pts_r_in[:,1], 'r.')

clf.fit_predict(pts_l)
scores_l = clf.negative_outlier_factor_
mask_l = np.abs(-1-scores_l) < 0.5
pts_l_in = pts_l[mask_l]

plt.figure()
plt.plot(pts_l[:,0], pts_l[:,1], 'b.')
plt.plot(pts_l_in[:,0], pts_l_in[:,1], 'r.')

In [ ]:
np.flip(np.arange(6).reshape(3,2), axis=0)e

# Stereo Reconstruction

## First Method : Bspline Cost function
Method:
1. Initialize 3-D stereo reconstruction
2. Project using stereo parameters
3. Measure error in 2-D bspline images.

In [ ]:
curv_dir = curvature_dir[3]
print(curv_dir)
rois_lr = np.load(curv_dir + 'rois_lr.npy')
img_num = 2
left_img = cv.imread(curv_dir + f"left-{img_num:04d}.png", cv.IMREAD_ANYCOLOR)
right_img = cv.imread(curv_dir + f"right-{img_num:04d}.png", cv.IMREAD_ANYCOLOR)
rois_l = rois_lr[:,:,0:2].tolist()
rois_r = rois_lr[:,:,2:4].tolist()
roi_l = tuple(rois_l[img_num])
roi_r = tuple(rois_r[img_num])


In [ ]:
def cost_proj3d(coeffs, pts_ref_l, pts_ref_r, stereo_params: dict, order: int ):
    ''' 2-D points should be of shape 2 X N'''
    # get the bspline 3-D points
    c = coeffs.reshape(3, order)
    bspline = BSpline3D(order, c)
    s = np.linspace(0, 1, pts_ref_l.shape[1])
    bspl_pts = bspline(s)
    
    # project each of the points
    bspl_proj_l = cv.projectPoints(bspl_pts, np.eye(3), np.zeros(3), stereo_params['cameraMatrix1'], stereo_params['distCoeffs1'])[0].squeeze().T
    bspl_proj_r = cv.projectPoints(bspl_pts, stereo_params['R'], stereo_params['t'], stereo_params['cameraMatrix2'], stereo_params['distCoeffs2'])[0].squeeze().T
    
    # calculate standard 3-D cost
    cost_per_pt = np.sum(np.abs(pts_ref_l - bspl_proj_l)**2, axis=1) + np.sum(np.abs(pts_ref_r - bspl_proj_r), axis=1)
    cost = np.sum(cost_per_pt)
    
    return cost
    

# cost_proj3d
    
def stereo_reconstruction(left_img, right_img, bor_l: list = [], bor_r: list = [],
                          roi_l: tuple = (), roi_r: tuple = (),
                          proc_show: bool = False):
    # Get the 2-D image skeletons
    left_skel, right_skel, conts_l, conts_r = imgproc_jig(left_img, right_img,
                                                                bor_l, bor_r, 
                                                                roi_l, roi_r,
                                                                proc_show)
    # fit a bspline to each contour (for interpolation)
    len_thresh = 5
    bspl_k = 3
    out_thresh = 1.25
    n_neigh = 20
    pts_l, bspline_l = centerline_from_contours( conts_l,
                                                 len_thresh = len_thresh,
                                                 bspline_k = bspl_k,
                                                 outlier_thresh = out_thresh,
                                                 num_neigbors = n_neigh )
    
    pts_r, bspline_r = centerline_from_contours( conts_r,
                                                 len_thresh = len_thresh,
                                                 bspline_k = bspl_k,
                                                 outlier_thresh = out_thresh,
                                                 num_neigbors = n_neigh )
    
    # get the pixel points in the images
    pts_px_l = np.unique(pts_l.astype(int), axis=0)
    pts_px_r = np.unique(pts_r.astype(int), axis=0)
    
    
    # perform an optimization over the bspline points (3-D to 2-D projection)
    
    
    
    return bspline_l, bspline_r, pts_l, pts_r
    
    
    
# stereo_reconstruction
    

In [ ]:
bspline_l, bspline_r, pts_l, pts_r = stereo_reconstruction(left_img, right_img, roi_l = roi_l, roi_r = roi_r, proc_show=False)
s = np.linspace(0,1,200)
pts_bspl_l = np.vstack((bspline_l.unscale(s), bspline_l.eval_unscale(s))).T
pts_bspl_r = np.vstack((bspline_r.unscale(s), bspline_r.eval_unscale(s))).T

pts_bspl_dis_l = np.unique()

In [ ]:
plt.figure()
plt.plot(pts_l[:,0], pts_l[:,1], '.')
plt.plot(pts_bspl_l[:,0], pts_bspl_l[:,1])
plt.ylim(0.9*min(pts_l[:,1]), 1.1*max(pts_l[:,1]))

plt.figure()
plt.plot(pts_r[:,0], pts_r[:,1], '.')
plt.plot(pts_bspl_r[:,0], pts_bspl_r[:,1])
plt.ylim(0.9*min(pts_r[:,1]), 1.1*max(pts_r[:,1]))

### Test out B-Spline fitting

In [ ]:
coeffs = np.arange(6).reshape(-1,1)
coeffs = np.eye(5)[0,:].reshape(-1,1)
x = [0,1]
s = [0,1]
bp = BPoly(coeffs, x)

print(bp.c.flatten())
bp(s)

In [ ]:
def bern_basis(order):
    C = np.eye(order)
    
    basis = []
    for i in range(order):
        coeff = C[i,:].reshape(-1,1)
        basis.append(BPoly(coeff, [0,1]))
    
    # for
    
    return basis

# bern_basis

def bern_call(bern_list, s):
    retval = []
    for bp in bern_list:
        retval.append(bp(s))
        
    return retval

# def

s = [0, 0.25, 0.5, 0.75, 1]
basis = bern_basis(3)
A = np.array(bern_call(basis, s))

pts = np.arange(3*len(s)).reshape(len(s), 3)
print(A.shape, pts.shape)

x, *_ = np.linalg.lstsq(A.T, pts, rcond=None)
print(x.shape,'\n', x.round(3))

print(np.round(A.T @ x, 3))
print(pts)
bp3 = BSpline3D(3, x.T)
bp3(s).round(3) - pts.T

Test the cost function and minimziation

In [ ]:
def cost_proj3d(coeffs, pts_ref_l, pts_ref_r, stereo_params: dict, order: int ):
    ''' 2-D points should be of shape 2 X N'''
    # get the bspline 3-D points
    c = coeffs.reshape(3, order)
    bspline = BSpline3D(order, c)
    s = np.linspace(0, 1, pts_ref_l.shape[1])
    bspl_pts = bspline(s)
    
    # project each of the points
    bspl_proj_l = cv.projectPoints(bspl_pts, np.eye(3), np.zeros(3), stereo_params['cameraMatrix1'], stereo_params['distCoeffs1'])[0].squeeze().T
    bspl_proj_r = cv.projectPoints(bspl_pts, stereo_params['R'], stereo_params['t'], stereo_params['cameraMatrix2'], stereo_params['distCoeffs2'])[0].squeeze().T
    
    # calculate standard 3-D cost
    cost_per_pt = np.sum((pts_ref_l - bspl_proj_l)**2, axis=1) + np.sum((pts_ref_r - bspl_proj_r)**2, axis=1)
    cost = np.sum(cost_per_pt)
    
    return cost
    

# cost_proj3d

# start testing
np.random.seed(110)
order = 3
coeffs = np.random.randn(3,order)
bp3 = BSpline3D(order, coeffs) # true function
s = np.linspace(0, 1, 80)
bspl_pts = bp3(s)

# cost function test
print('Test cost function')

left_proj = cv.projectPoints(bspl_pts, np.eye(3), np.zeros(3), stereo_params['cameraMatrix1'], stereo_params['distCoeffs1'])[0].squeeze().T
right_proj = cv.projectPoints(bspl_pts, stereo_params['R'], stereo_params['t'], stereo_params['cameraMatrix2'], stereo_params['distCoeffs2'])[0].squeeze().T

print('Cost = ', cost_proj3d(bp3.coeffs.flatten(), left_proj, right_proj, stereo_params, order))


# test optimization
print('\nTest optimization')

coeffs_init = coeffs + np.random.randn(*coeffs.shape)/25

res = minimize(cost_proj3d, coeffs_init.flatten(), args=(left_proj, right_proj, stereo_params, order))
print('Optimal Cost', res.fun)
print(res.x, '\n', coeffs)
bp3_opt = BSpline3D(order, res.x.reshape(3, order))
bp3_pts = bp3(s)
bp3_opt_pts = bp3_opt(s)

print(bp3_pts.shape, bp3_opt_pts.shape, s.shape)

fig = plt.figure(figsize=(12,8))
ax = plt.axes(projection='3d')
ax.plot3D(bp3_pts[0,:], bp3_pts[1,:],bp3_pts[2,:])
ax.plot3D(bp3_opt_pts[0,:], bp3_opt_pts[1,:],bp3_opt_pts[2,:],'-.')
plt.legend(['actual', 'fit'])

In [ ]:
a = np.random.randn(5,3)
print(np.sum(a, axis=1).shape, np.sum(a, axis=0).shape)

## Second Method: Stereo Rectification
Using stereoRectification and then disparity mapping

In [ ]:
def stereo_rectify2(img_left, img_right, stereo_params, interp_method = cv.INTER_LINEAR, alpha:float = -1, 
                   force_recalc: bool=False):
    # gray-scale the image
    left_gray = cv.cvtColor(img_left, cv.COLOR_BGR2GRAY)
    right_gray = cv.cvtColor(img_right, cv.COLOR_BGR2GRAY)
    
    # perform stereo rectification
    K_l = stereo_params['cameraMatrix1'].astype(float)
    K_r = stereo_params['cameraMatrix2'].astype(float)
    dists_l = stereo_params['distCoeffs1'].astype(float)
    dists_r = stereo_params['distCoeffs2'].astype(float)
    R = stereo_params['R'].astype(float)
    t = stereo_params['t'].astype(float)
    
    if force_recalc or any(k not in stereo_params.keys() for k in ['R1', 'R2', 'P1', 'P2']):
        R_l, R_r, P_l, P_r, Q, roi_l, roi_r = cv.stereoRectify(K_l, dists_l, K_r, dists_r, left_gray.shape[::-1], 
                                                 R, t, alpha=alpha)
        stereo_params['R1'] = R_l
        stereo_params['R2'] = R_r
        stereo_params['P1'] = P_l
        stereo_params['P2'] = P_r
        stereo_params['Q'] = Q
        rois_lr = (roi_l, roi_r)
        
    # if
    
    else:
        R_l = stereo_params['R1']
        R_r = stereo_params['R2']
        P_l = stereo_params['P1']
        P_r = stereo_params['P2']
        rois_lr = tuple(2*[[[0, 0], [-1, 1]]])
        
    # else
    
    print(P_l, '\n', P_r, end='\n\n')
    print(P_l - P_r)
    
    # compute stereo rectification map
    map1_l, map2_l = cv.initUndistortRectifyMap(K_l, dists_l, R_l, P_l, left_gray.shape[::-1], cv.CV_32FC1)
    map1_r, map2_r = cv.initUndistortRectifyMap(K_r, dists_r, R_r, P_r, right_gray.shape[::-1], cv.CV_32FC1)
    
    # apply stereo rectification map
    img_left = roi(img_left, roi_l, full=True)
    img_right = roi(img_right, roi_r, full=True)
    left_rect = cv.remap(img_left, map1_l, map2_l, interp_method)
    right_rect = cv.remap(img_right, map1_r, map2_r, interp_method)
    
    return left_rect, right_rect, rois_lr, (map1_l, map2_l), (map1_r, map2_r)
    
# stereo_rectify

In [ ]:
# set-up
roi_all = []
roi_l = [[0, 210], [-1, 775]]
roi_r = [[0, roi_l[0][1] + 25], [-1, roi_l[1][1] + 25]]

# roi images
left_roi = roi(left_img, roi_l, full=True)
right_roi = roi(right_img, roi_r, full=True)

# stereo rectification
left_rect, right_rect, valid_roi_lr, *maps = stereo_rectify(left_roi, right_roi, stereo_params, cv.INTER_LINEAR,
                                                           force_recalc=True, alpha=0.5)

print(left_img.shape, right_img.shape)
print(left_rect.shape, right_rect.shape)

# plotting
plt.figure(figsize=(12,8))
plt.imshow(imconcat(left_img, right_img, [255, 0, 0]),cmap='gray')
plt.title('Original Images')

plt.figure(figsize=(12,8))
plt.imshow(imconcat(left_roi, right_roi, [255, 0, 0]),cmap='gray')
plt.title('ROI: Original Images')

plt.figure(figsize=(12,8))
plt.imshow(imconcat(left_rect, right_rect, [255,0, 0]))
plt.title('Rectified Images')



print('validROI:', type(valid_roi_lr), len(valid_roi_lr), valid_roi_lr[0], valid_roi_lr[1] )


In [ ]:
# set-up
roi_all = []
roi_l = [[0, 210], [-1, 775]]
roi_r = [[0, roi_l[0][1] + 25], [-1, roi_l[1][1] + 25]]

# roi images
left_roi = roi(left_img, roi_l, full=True)
right_roi = roi(right_img, roi_r, full=True)

#test the roi after rectification
left_rect, right_rect, valid_roi_lr, map_l, map_r = stereo_rectify(left_roi, right_roi, stereo_params, cv.INTER_LINEAR,
                                                           force_recalc=True, alpha=0.5)

# map the ROIS
roi_l_img = roi_image(roi_l, left_img.shape)
roi_r_img = roi_image(roi_r, right_img.shape)

roi_l_mapped = cv.remap(roi_l_img.astype(np.uint8), map_l[0], map_l[1], cv.INTER_NEAREST)
roi_r_mapped = cv.remap(roi_r_img.astype(np.uint8), map_r[0], map_r[1], cv.INTER_NEAREST)

print(roi_l_mapped.min(), roi_l_mapped.max(), np.unique(roi_l_mapped))
print(roi_r_mapped.min(), roi_r_mapped.max(), np.unique(roi_r_mapped))


# check the rois
left_rect_roi_check = left_rect.copy()
right_rect_roi_check = right_rect.copy()
left_rect_roi_check[roi_l_mapped == 0] = [0, 0, 255]
right_rect_roi_check[roi_r_mapped == 0] = [0, 0, 255]

# plot the results
plt.figure(figsize=(12,8))
plt.imshow(imconcat(left_rect, right_rect, [255,0, 0]))
plt.title('Rectified Images')

plt.figure(figsize=(12,8))
plt.imshow(imconcat(roi_l_mapped, roi_r_mapped, 0), cmap='gray')
plt.title('Mapped rois')

plt.figure(figsize=(12,8))
plt.imshow(imconcat(left_rect_roi_check, right_rect_roi_check, [255,0, 0]))
plt.title('Rectified Images: out of roi in blue')



### Stereo Reconstruction Method

Procedure:
1. Rectify input images using `stereo_rectify` function.
2. From the rectified images, determine the location of the needle using image processing
3. Using the locations of the needle, perform a "disparity" triangulation algorithm. 
    - This will most likely use `cv.triangulatePoints`

In [ ]:
def stereomatch_needle2( left_conts, right_conts, method = "tip-count", col:int = 1,
                        bspline_l = None, bspline_r = None ):
    ''' stereo matching needle arclength points for the needle
        
        
        Args:
            (left/right)_conts: a nx2 array of pixel coordinates
                                for the contours in the (left/right) image
            
            start_location (Default: "tip"): a string of where to start counting.
                                             tip is only implemented.
                                             
            col (int = 1): the column to begin matching by
    
     '''
    # squeeze dimensions just in case
    left_conts = np.squeeze( left_conts )
    right_conts = np.squeeze( right_conts )
    
    # remove duplicate rows
    pts_l = np.unique( left_conts, axis = 0 )
    pts_r = np.unique( right_conts, axis = 0 )
    
    # find the minimum number of points to match
    
    if method == "tip-count":
        if bspline_l is None or bspline_r is None:
            n = min( pts_l.shape[0], pts_r.shape[0] )
            left_idx = np.argsort( pts_l[:, col] )[-n:]
            right_idx = np.argsort( pts_r[:, col] )[-n:]
            
            left_matches = pts_l[left_idx]
            right_matches = pts_r[right_idx]
            
        # if
        
        else:
            warnings.filterwarnings( 'ignore', category = UserWarning, module = 'BSpline1D' )
            # determine each of the bspline arclengths
            s = np.linspace(0, 1, 200)
            pts_lx = bspline_l._unscale(s)
            pts_ly = bspline_l(s)
            
            pts_rx = bspline_r._unscale(s)
            pts_ry = bspline_r(s)
            if col == 0:
                pts_l = np.vstack((pts_lx, pts_ly)).T
                pts_r = np.vstack((pts_rx, pts_ry)).T
                
            # if
            
            else:
                pts_l = np.vstack((pts_lx, pts_ly)).T
                pts_r = np.vstack((pts_rx, pts_ry)).T
                
            # else
            
            left_matches = pts_l
            right_matches = pts_r
            
        # else
        
    # if
    
    elif method == 'disparity': 
        if bspline_l is None or bspline_r is None:
            # match from tip (bottom most point | largest number)
            uniq_ax = np.unique(np.append(pts_l[:,col], pts_r[:,col])) # unique values in the columns
            
            left_matches = np.zeros((0,2))
            right_matches = np.zeros((0,2))
            for val in uniq_ax:
                # find the rows that have this value
                mask_val_l = pts_l[:,col] == val
                mask_val_r = pts_l[:,col] == val
                
                # if the both have matches, add the means to the matches
                if np.any(mask_val_l) and np.any(mask_val_r):
                    match_l = np.mean(pts_l[mask_val_l, :], axis=0, keepdims=True)
                    match_r = np.mean(pts_r[mask_val_r, :], axis=0, keepdims=True)
                    
                    left_matches = np.append(left_matches, match_l, axis=0)
                    right_matches = np.append(right_matches, match_r, axis=0)
                
            # for
        # if
        
        else:
#             s1 = np.linspace(0, 1, 200) 
#             y1_l = bspline_l._unscale(s1)
#             y1_r = bspline_r._unscale(s2)
            
            # determine bounds on y-axis
            y_min = max(bspline_l.qmin, bspline_r.qmin)
            y_max = min(bspline_r.qmax, bspline_r.qmax)
            
            y = np.linspace(y_min, y_max, 200)
            
            # get the x-coordinates of each b-spline
            left_matches = np.vstack((bspline_l(y), y)).T
            right_matches = np.vstack((bspline_r(y), y)).T
            
            
    # elif
    
    else:
        raise NotImplementedError( f"method = {method} not implemented." )
    
    # else
    
    return left_matches, right_matches

# stereomatch_needle2

def centerline_from_contours2( contours, len_thresh:int = -1, bspline_k:int = -1,
                               outlier_thresh:float = -1, num_neigbors: int = -1 ):
    ''' This is to determine the centerline points from the contours using outlier detection
        and bspline smoothing
        
        bspline fits on the y-axis as the independent variable (where the stereo-matching occurs)
    
        @param contours: the image contours
        @param len_thresh: a length threshold to remove small length contours.
                                             using np.inf will filter out all but the longest length
                                             contour.
        @param bspline_k: the degree to fit a bspline. If less than 1, bspline will not be fit and
                          will return bspline=None.
        
        @param outlier_thresh: the outlier thresholding value
        @param num_neighbors: the number of neigbhors parameter needed for the outlier detection
        
        @return: pts, bspline:
                    pts = [N x 2] numpy array of bspline pts (i, j) image points
                    bspline = None or bspline that was fit to the contours 
    
    '''
    # length thresholding
    len_thresh = min( len_thresh, max( [len( c ) for c in contours] ) )  # don't go over max length
    contours_filt = [c for c in contours if len( c ) >= len_thresh]
    
    # numpy-tize points
    pts = np.unique( np.vstack( contours_filt ).squeeze(), axis = 0 )
    
    # outlier detection
    if ( outlier_thresh >= 0 ) and ( num_neigbors > 0 ):
        clf = LocalOutlierFactor( n_neighbors = num_neigbors, contamination = 'auto' )
        clf.fit_predict( pts )
        inliers = np.abs( -1 - clf.negative_outlier_factor_ ) < outlier_thresh

        pts = pts[inliers]
        
    # if: outlier detection
    
    # fit bspline
    bspline = None
    if bspline_k > 0:
#         set_trace()
        idx = np.argsort(pts[:,1])
        bspline = BSpline1D( pts[idx, 1], pts[idx, 0], k = bspline_k )
        
        # grab all of the bspline points
        s = np.linspace( 0, 1, 200 )
        pts = np.stack( ( bspline.unscale( s ), bspline( bspline.unscale( s ) ) ) ).T[:, [1,0]]
        
    # if: bspline

    return pts, bspline

# centerline from_contours2

In [ ]:
def needle_reconstruction2(img_left, img_right, stereo_params, 
                           bor_l:list = [], bor_r:list = [],
                           roi_l = (), roi_r = (), 
                           alpha: float = 0.5, recalc_stereo:bool = False,
                           proc_show: bool = False):
    ''' This is function for performing a 3-D needle reconstruction from a raw stereo pair'''
    # prepare images
    ## roi the images
    left_roi = roi(img_left, roi_l, full=True)
    right_roi = roi(img_right, roi_r, full=True)
    
    ## black-out regions
    left_roibo = blackout_regions(left_roi, bor_l)
    right_roibo = blackout_regions(right_roi, bor_r)
    
    # stereo rectify the images
    left_rect, right_rect, _, map_l, map_r = stereo_rectify(left_roibo, right_roibo, stereo_params, 
                                                                       interp_method = cv.INTER_LINEAR, alpha=alpha, 
                                                                       force_recalc = recalc_stereo)

    ## map the rois
    boroi_l_img = roi_image(roi_l, left_img.shape) & blackout_image(bor_l, left_img.shape)
    boroi_r_img = roi_image(roi_r, right_img.shape) & blackout_image(bor_r, right_img.shape)
    
    boroi_l_mapped = cv.remap(boroi_l_img.astype(np.uint8), map_l[0], map_l[1], cv.INTER_NEAREST)
    boroi_r_mapped = cv.remap(boroi_r_img.astype(np.uint8), map_r[0], map_r[1], cv.INTER_NEAREST)
    
    # perform image processing on rectified images
    left_gray = cv.cvtColor(left_rect, cv.COLOR_BGR2GRAY)
    right_gray = cv.cvtColor(right_rect, cv.COLOR_BGR2GRAY)
    left_thresh, right_thresh = thresh(left_gray, right_gray, thresh=50)
    
    ## remove extra borders threshed out
    left_thresh *= boroi_l_mapped.astype(np.uint8)
    right_thresh *= boroi_r_mapped.astype(np.uint8)
    
    # get the contours and filter out outliers
    left_skel, right_skel = skeleton(left_thresh, right_thresh)
    conts_l, conts_r = contours(left_skel, right_skel)
    
    ## outlier options
    len_thresh = 5
    bspl_k = 2
    out_thresh = 1.25
    n_neigh = 60
   
    pts_l, bspline_l = centerline_from_contours2( conts_l,
                                                  len_thresh = len_thresh,
                                                  bspline_k = bspl_k,
                                                  outlier_thresh = out_thresh,
                                                  num_neigbors = n_neigh )
    
    pts_r, bspline_r = centerline_from_contours2( conts_r,
                                                  len_thresh = len_thresh,
                                                  bspline_k = bspl_k,
                                                  outlier_thresh = out_thresh,
                                                  num_neigbors = n_neigh )
    
    # stereo matching
    pts_l_match, pts_r_match = stereomatch_needle2(pts_l, pts_r, method='disparity', 
                                                   bspline_l=bspline_l, bspline_r=bspline_r)
    
    # stereo triangulation
    pts_3d = cv.triangulatePoints(stereo_params['P1'], stereo_params['P2'], pts_l_match.T, pts_r_match.T)
    pts_3d /= pts_3d[-1,:]
    pts_3d = pts_3d.T
       
    
    
    # show processing
    if proc_show:
        plt.figure(figsize=(12,8))
        plt.imshow(imconcat(left_rect, right_rect, [255,0, 0]))
        plt.title('Rectified Images')
        
        plt.figure(figsize=(12,8))
        plt.imshow(imconcat(left_thresh, right_thresh, 125), cmap='gray')
        plt.title('Thresholded rectified images')
        
        plt.figure(figsize=(12,8))
        plt.imshow(imconcat(left_skel, right_skel, 125), cmap='gray')
        plt.title('Skeletonized threshold')
        
        left_rect_draw = cv.polylines(left_rect.copy(), [pts_l.reshape(-1,1,2).astype(np.int32)], False, (255, 0, 0), 5)
        right_rect_draw = cv.polylines(right_rect.copy(), [pts_r.reshape(-1,1,2).astype(np.int32)], False, (255, 0, 0), 5)
        plt.figure(figsize=(12,8))
        plt.imshow(imconcat(left_rect_draw, right_rect_draw, [255, 0, 0]))
        plt.title('centerline points')
        
        fig = plt.figure(figsize=(8,8))
        plt.plot(pts_l_match[:,0] - pts_r_match[:,0])
        plt.title('disparity')
        
        fig = plt.figure(figsize=(8,8))
        ax = plt.gca(projection='3d')
        ax.plot(pts_3d[:,0], pts_3d[:,1], pts_3d[:,2])
        plt.title('3-D reconstruction')
        

    # if
    
    
#     return left_rect, right_rect, left_thresh, right_thresh, boroi_l_mapped, boroi_r_mapped
    return pts_3d, pts_l, pts_r, bspline_l, bspline_r
    
# needle_reconstruction2


# test function 
## set-up
roi_all = []
roi_l = [[0, 210], [-1, 700]]
roi_r = [[roi_l[0][0], roi_l[0][1] + 25], [roi_l[1][0], roi_l[1][1] + 25]]

# l_rect, r_rect, l_ret, r_ret, borm_l, borm_r = needle_reconstruction2(left_img, right_img, stereo_params, roi_l=roi_l, roi_r=roi_r, alpha=0.5, recalc_stereo=True, proc_show=True)
pts_3d, pts_l, pts_r, bspline_l, bspline_r = needle_reconstruction2(left_img, right_img, stereo_params, 
                                                            roi_l=roi_l, roi_r=roi_r, 
                                                            alpha=0.5, recalc_stereo=True, proc_show=True)


pts_l_match, pts_r_match = stereomatch_needle2(pts_l, pts_r, method='disparity', bspline_l=bspline_l, bspline_r=bspline_r)

plt.figure(figsize=(8,8))
plt.plot(pts_l[:,0], pts_l[:,1], 'k-', linewidth=6)
plt.plot(pts_l_match[:,0], pts_l_match[:,1], 'g-', linewidth=3)
plt.title('left matches')
plt.legend(['points determined', 'matched points'])

plt.figure(figsize=(8,8))
plt.plot(pts_r[:,0], pts_r[:,1], 'k-', linewidth=6)
plt.plot(pts_r_match[:,0], pts_r_match[:,1], 'g-', linewidth=3)
plt.title('right matches')
plt.legend(['points determined', 'matched points'])

plt.figure(figsize=(8,8))
plt.plot(pts_l_match[:,0], pts_l_match[:,1],  linewidth=3)
plt.plot(pts_r_match[:,0], pts_r_match[:,1],  linewidth=3)
plt.title('left-right matches')
plt.legend(['left', 'right'])

In [ ]:
pts_3d_h = pts_3d / pts_3d[:,-1].reshape(-1,1)
pts_3d_h[:, 0:-1] -= np.mean(pts_3d_h[:,0:-1])
print(pts_3d[0:2])
print(pts_3d_h[0:2])

fig = plt.figure(figsize=(8,8))
ax = plt.gca(projection='3d')
ax.plot(pts_3d_h[:,0], pts_3d_h[:,1], pts_3d_h[:,2])

fig = plt.figure(figsize=(8,8))
plt.plot(pts_3d_h[:,0], label='x')
plt.plot(pts_3d_h[:,1], label='y')
plt.plot(pts_3d_h[:,2], label='z')
plt.legend()

np.savetxt('../test_3d_triangulation.txt', pts_3d_h)

In [ ]:
# test opencv's triangulate points function
points = np.random.randn(4,10)
points[-1,:] = 1

print(points[:,:4], end='\n\n')

points_l_uh = stereo_params['P1'] @ points
points_l = points_l_uh[:-1,:]/points_l_uh[-1,:]

points_r_uh = stereo_params['P2'] @ points
points_r = points_r_uh[:-1,:]/points_r_uh[-1,:]

print(points_l[:,:4], end='\n\n')
print(points_r[:,:4], end='\n\n')

tri_points = cv.triangulatePoints(stereo_params['P1'], stereo_params['P2'], points_l, points_r)

print(tri_points[:,:4], end='\n\n')

tri_points_h = tri_points[:,:]/tri_points[-1,:]

print(tri_points_h[:,:4], end='\n\n')

dist = np.linalg.norm(tri_points_h - points, axis=0)


plt.plot(dist)
plt.hlines(0.5, 0, len(dist)-1, 'r', '--')

### Stereo Matching: More Robust

#### Stereo Matching: NormxCorr

In [53]:
def stereomatch_needle3( left_conts, right_conts, img_left, img_right, 
                         roi_l_mask = None, roi_r_mask = None, score_thresh = 0.75, # thresh=50,
                         col:int = 1, zoom=1.0, winsize=(5,5) ):
    ''' stereo matching needle arclength points for the needle
        
        performs this using normxcorr along 'col'-axis
        
        Args:
            (left/right)_conts: a nx2 array of pixel coordinates
                                for the contours in the (left/right) image
            
            img_(left/right): the left and right rectified images.
            
            roi_(left/right): the rectified left/right ROI images.
                                             
            col (int = 1): the column to begin matching by
    
     '''
    assert((0 < zoom) and (zoom < 3)) # limit the zoom to (0, 3)
    
    assert(len(winsize) == 2)
    assert(all(length % 2 == 1 for length in winsize)) # make sure they are all both odd 
    
    # orocess the contours
    if isinstance(left_conts, np.ndarray):
        left_conts = np.squeeze( left_conts ) # squeeze dimensions just in case
    
    # if
    
    elif isinstance(left_conts, BSpline1D):
        s = np.linspace(0, 1, 200)
        if col == 0:
            left_conts = np.vstack( ( bspline_l._unscale(s), bspline_l(s) ) ).T
            
        else:
            left_conts = np.vstack( ( bspline_l(s), bspline_l._unscale(s) ) ).T
            
    # elif
    
    else:
        raise TypeError('left_conts is not a BSpline1D or numpy array.')
        
    # else
    
    if isinstance(right_conts, np.ndarray):
        right_conts = np.squeeze( right_conts ) # squeeze dimensions just in case
    
    # if
    
    elif isinstance(right_conts, BSpline1D):
        s = np.linspace(0, 1, 200)
        if col == 0:
            right_conts = np.vstack( ( bspline_r._unscale(s), bspline_r(s) ) ).T
            
        else:
            right_conts = np.vstack( ( bspline_r(s), bspline_r._unscale(s) ) ).T
            
    # elif
    
    else:
        raise TypeError('right_conts is not a BSpline1D or numpy array.')
        
    # else
        
    # remove duplicate rows
    pts_l = np.unique( left_conts.round().astype(np.int32), axis = 0 )
    pts_r = np.unique( right_conts.round().astype(np.int32), axis = 0 )
    
    # helper functions
    def normxcorr1(template, img, row, img_mask = None):
        ''' returns the best match in a line along a specific index'''
        # unpack template shape
        t_0, t_1 = template.shape

        # get the search image
        img_search = img[row-t_0//2:row+t_0//2+1, :]
        set_trace()
        res = cv.matchTemplate(img_search, template, cv.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
        
        best_idx = (max_loc[1] + row, max_loc[0] + t_1//2)
        
        return best_idx, max_val
                
    # def: normxcorr1
    
    # match points left-to-right
#     winsize = tuple(reversed(winsize))
    left_matches = np.zeros((0, 2))
    right_matches = np.zeros((0,2))
    for i in range(pts_l.shape[0]):
        # grab the template
        px_pt = np.flip(pts_l[i]).round().astype(int)
        
        img_roi = np.vstack((-np.array(winsize)//2, np.array(winsize)//2 + 1)) + px_pt.reshape(1,-1).tolist()
        img_roi[img_roi < 0] = 0
#         img_roi = np.flip(img_roi, axis=1)
        
        template = roi(img_left, img_roi, full=False) 
        
        if template.shape[0] % 2 == 0:
            template = template[:-1,:]

        if template.shape[1] % 2 == 0:
            template = template[:,:-1]
        
#         set_trace()
        
        match_pt, score = normxcorr1(template, img_right, px_pt[0])
        
        if score > score_thresh:
            left_matches = np.append(left_matches, np.reshape(px_pt, (1,2)), axis=0)
            right_matches = np.append(right_matches, np.reshape(match_pt, (1,2)), axis=0)
            
        # if
        
#         break 
    # for
    
    return left_matches, right_matches
    
# stereomatch_needle3

In [ ]:
def needle_reconstruction3(img_left, img_right, stereo_params, 
                           bor_l:list = [], bor_r:list = [],
                           roi_l = (), roi_r = (), 
                           alpha: float = 0.5, recalc_stereo:bool = False,
                           proc_show: bool = False):
    ''' This is function for performing a 3-D needle reconstruction from a raw stereo pair'''
    # prepare images
    ## roi the images
    left_roi = roi(img_left, roi_l, full=True)
    right_roi = roi(img_right, roi_r, full=True)
    
    
    ## black-out regions
    left_roibo = blackout_regions(left_roi, bor_l)
    right_roibo = blackout_regions(right_roi, bor_r)
    imgs_ret = {'roibo': (left_roibo, right_roibo)}
    
    # stereo rectify the images
    left_rect, right_rect, _, map_l, map_r = stereo_rectify(left_roibo, right_roibo, stereo_params, 
                                                                       interp_method = cv.INTER_LINEAR, alpha=alpha, 
                                                                       force_recalc = recalc_stereo)
    imgs_ret['rect'] = (left_rect, right_rect)
    
    ## map the rois
    boroi_l_img = roi_image(roi_l, left_img.shape) & blackout_image(bor_l, left_img.shape)
    boroi_r_img = roi_image(roi_r, right_img.shape) & blackout_image(bor_r, right_img.shape)
    imgs_ret['boroi-bool'] = (boroi_l_img, boroi_r_img)
    
    boroi_l_mapped = cv.remap(boroi_l_img.astype(np.uint8), map_l[0], map_l[1], cv.INTER_NEAREST)
    boroi_r_mapped = cv.remap(boroi_r_img.astype(np.uint8), map_r[0], map_r[1], cv.INTER_NEAREST)
    imgs_ret['boroi-bool-mapped'] = (boroi_l_mapped, boroi_r_mapped)
    
    # perform image processing on rectified images
    left_gray = cv.cvtColor(left_rect, cv.COLOR_BGR2GRAY)
    right_gray = cv.cvtColor(right_rect, cv.COLOR_BGR2GRAY)
    imgs_ret['gray-rect'] = (left_gray, right_gray)
    left_thresh, right_thresh = thresh(left_gray, right_gray, thresh=50)
    
    ## remove extra borders threshed out
    left_thresh *= boroi_l_mapped.astype(np.uint8)
    right_thresh *= boroi_r_mapped.astype(np.uint8)
    imgs_ret['thresh-rect'] = (left_thresh, right_thresh)
    
    # get the contours and filter out outliers
    left_skel, right_skel = skeleton(left_thresh, right_thresh)
    imgs_ret['skel'] = (left_skel, right_skel)
    conts_l, conts_r = contours(left_skel, right_skel)
    
    ## outlier options
    len_thresh = 5
    bspl_k = 2
    out_thresh = 1.25
    n_neigh = 60
   
    pts_l, bspline_l = centerline_from_contours( conts_l,
                                                  len_thresh = len_thresh,
                                                  bspline_k = bspl_k,
                                                  outlier_thresh = out_thresh,
                                                  num_neigbors = n_neigh )
    
    pts_r, bspline_r = centerline_from_contours( conts_r,
                                                  len_thresh = len_thresh,
                                                  bspline_k = bspl_k,
                                                  outlier_thresh = out_thresh,
                                                  num_neigbors = n_neigh )
    
    # stereo matching
    left_rect_gray = cv.cvtColor(left_rect, cv.COLOR_BGR2GRAY)
    right_rect_gray = cv.cvtColor(right_rect, cv.COLOR_BGR2GRAY)
    pts_l_match, pts_r_match = stereomatch_needle3( pts_l, pts_r, left_rect_gray, right_rect_gray, winsize=(101,101) )
    
#     print(np.hstack((pts_l_match, pts_r_match)))
    
    # stereo 
    pts_3d = None
#     pts_3d = cv.triangulatePoints(stereo_params['P1'], stereo_params['P2'], pts_l_match.T, pts_r_match.T)
#     pts_3d /= pts_3d[-1,:]
#     pts_3d = pts_3d.T
       
    
    
    # show processing
    if proc_show:
        plt.figure(figsize=(12,8))
        plt.imshow(imconcat(left_rect, right_rect, [255,0, 0]))
        plt.title('Rectified Images')
        
        plt.figure(figsize=(12,8))
        plt.imshow(imconcat(left_thresh, right_thresh, 125), cmap='gray')
        plt.title('Thresholded rectified images')
        
        plt.figure(figsize=(12,8))
        plt.imshow(imconcat(left_skel, right_skel, 125), cmap='gray')
        plt.title('Skeletonized threshold')
        
        left_rect_draw = cv.polylines(left_rect.copy(), [pts_l.reshape(-1,1,2).astype(np.int32)], False, (255, 0, 0), 5)
        right_rect_draw = cv.polylines(right_rect.copy(), [pts_r.reshape(-1,1,2).astype(np.int32)], False, (255, 0, 0), 5)
        plt.figure(figsize=(12,8))
        plt.imshow(imconcat(left_rect_draw, right_rect_draw, [255, 0, 0]))
        plt.title('centerline points')

        left_rect_draw = cv.polylines(left_rect.copy(), [pts_l_match.reshape(-1,1,2).astype(np.int32)], False, (255, 0, 0), 5)
        right_rect_draw = cv.polylines(right_rect.copy(), [pts_r_match.reshape(-1,1,2).astype(np.int32)], False, (255, 0, 0), 5)
        plt.figure(figsize=(12,8))
        plt.imshow(imconcat(left_rect_draw, right_rect_draw, [255, 0, 0]))
        plt.title('matched centerline points')
        
        fig = plt.figure(figsize=(8,8))
        plt.plot(pts_l_match[:,0] - pts_r_match[:,0])
        plt.title('disparity')
        
#         fig = plt.figure(figsize=(8,8))
#         ax = plt.gca(projection='3d')
#         ax.plot(pts_3d[:,0], pts_3d[:,1], pts_3d[:,2])
#         plt.title('3-D reconstruction')
        

    # if
    
    
#     return left_rect, right_rect, left_thresh, right_thresh, boroi_l_mapped, boroi_r_mapped
    return pts_3d, pts_l, pts_r, bspline_l, bspline_r, imgs_ret
    
# needle_reconstruction2

# test function 
## set-up
roi_all = []
roi_l = [[0, 210], [-1, 700]]
roi_r = [[roi_l[0][0], roi_l[0][1] + 25], [roi_l[1][0], roi_l[1][1] + 25]]

# l_rect, r_rect, l_ret, r_ret, borm_l, borm_r = needle_reconstruction2(left_img, right_img, stereo_params, roi_l=roi_l, roi_r=roi_r, alpha=0.5, recalc_stereo=True, proc_show=True)
pts_3d, pts_l, pts_r, bspline_l, bspline_r, imgs = needle_reconstruction3(left_img, right_img, stereo_params, 
                                                                                roi_l=roi_l, roi_r=roi_r, 
                                                                                alpha=0.5, recalc_stereo=True, proc_show=True)
plt.figure()
plt.imshow(imgs['gray-rect'][0], cmap='gray')

> <ipython-input-53-1b39b0d678ef>(78)normxcorr1()
     76         img_search = img[row-t_0//2:row+t_0//2+1, :]
     77         set_trace()
---> 78         res = cv.matchTemplate(img_search, template, cv.TM_CCOEFF_NORMED)
     79         min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
     80 



ipdb>  template.shape


(101, 101)


ipdb>  img_search.shape


(101, 1024)


ipdb>  cv.matchTemplate(img_search, template, cv.TM_CCOEFF_NORMED)


array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+0

In [ ]:
winsize = (200, 600)
winsize = winsize
pts_li = pts_l.round().astype(int)
px_pt = np.array(left_img.shape[:2])//2
print(px_pt)

img_roi = np.vstack((-np.array(winsize)//2, np.array(winsize)//2 + 1)) + px_pt.reshape(1,-1).tolist()
img_roi[img_roi < 0] = 0
# img_roi = np.flip(img_roi, axis=1)

print(img_roi)

tmp = roi(imgs['gray-rect'][0], img_roi, full=False)
# ensure odd dimensions
if tmp.shape[0] % 2 == 0:
    tmp = tmp[:-1,:]
    
if tmp.shape[1] % 2 == 0:
    tmp = tmp[:,:-1]

print(tmp.shape)

plt.figure()
plt.imshow(imgs['gray-rect'][0], cmap='gray')
plt.figure()
plt.imshow(tmp,cmap='gray', vmin=0)

In [ ]:
x = np.zeros((0,2))
for i in range(3):
    x = np.append(x, np.reshape([i, i], (1,2)), axis=0)
    
x